In [139]:
import pymc
import numpy as np
import pandas as pd
import xarray as xr
import copy
import matplotlib.pyplot as plt
import pygemfxns_ensemble_sampling as es
import glob
import os
import run_calibration_list_multiprocess as rc

In [20]:
filelist = glob.glob(os.path.join('*.pickle'))
print(filelist)
traces = {}
for f in filelist:
    samples = f[:-20]
    name = f[-12:-7]
    print(samples, name)
    traces[(name, samples)] = pymc.database.pickle.load(f)

['10000Samples_10079.pickle', '3000Samples_10059.pickle', '5000Samples_10075.pickle', '30000Samples_10075.pickle', '5000Samples_10079.pickle', '10000Samples_10060.pickle', '30000Samples_10059.pickle', '30000Samples_10060.pickle', '3000Samples_10060.pickle', '30000Samples_10079.pickle', '10000Samples_10075.pickle', '1000Samples_10060.pickle', '1000Samples_10059.pickle', '5000Samples_10059.pickle', '5000Samples_10060.pickle', '1000Samples_10075.pickle', '1000Samples_10079.pickle', '10000Samples_10059.pickle', '3000Samples_10075.pickle', '3000Samples_10079.pickle']
10000 10079
3000 10059
5000 10075
30000 10075
5000 10079
10000 10060
30000 10059
30000 10060
3000 10060
30000 10079
10000 10075
1000 10060
1000 10059
5000 10059
5000 10060
1000 10075
1000 10079
10000 10059
3000 10075
3000 10079


In [21]:
traces

{('10079', '10000'): <pymc.database.pickle.Database at 0x7f1402a74390>,
 ('10059', '3000'): <pymc.database.pickle.Database at 0x7f1402a806a0>,
 ('10075', '5000'): <pymc.database.pickle.Database at 0x7f1402a80b00>,
 ('10075', '30000'): <pymc.database.pickle.Database at 0x7f1402a80f60>,
 ('10079', '5000'): <pymc.database.pickle.Database at 0x7f1402a80f98>,
 ('10060', '10000'): <pymc.database.pickle.Database at 0x7f1402a898d0>,
 ('10059', '30000'): <pymc.database.pickle.Database at 0x7f1402a89cc0>,
 ('10060', '30000'): <pymc.database.pickle.Database at 0x7f1402a89cf8>,
 ('10060', '3000'): <pymc.database.pickle.Database at 0x7f1402a74b38>,
 ('10079', '30000'): <pymc.database.pickle.Database at 0x7f1402a16630>,
 ('10075', '10000'): <pymc.database.pickle.Database at 0x7f1402a16a90>,
 ('10060', '1000'): <pymc.database.pickle.Database at 0x7f1402a16ef0>,
 ('10059', '1000'): <pymc.database.pickle.Database at 0x7f1402a16f28>,
 ('10059', '5000'): <pymc.database.pickle.Database at 0x7f1402a1f7f0>,

In [131]:
filelist = glob.glob(os.path.join('../MCMC_data/' , '*.nc'))
print(filelist)
ensembles = {}
samples = []
for f in filelist:
    print(f)
    name = f[38:-32]
    samples.append(name)
    print(name)
    ds = xr.open_dataset(f)
    ensembles[name] = ds

['../MCMC_data/parameter_sets_4glaciers_10000samples_200ensembles_20180727.nc', '../MCMC_data/parameter_sets_4glaciers_5000samples_200ensembles_20180727.nc', '../MCMC_data/parameter_sets_4glaciers_1000samples_200ensembles_20180727.nc', '../MCMC_data/parameter_sets_4glaciers_30000samples_200ensembles_20180727.nc', '../MCMC_data/parameter_sets_4glaciers_3000samples_200ensembles_20180727.nc']
../MCMC_data/parameter_sets_4glaciers_10000samples_200ensembles_20180727.nc
10000
../MCMC_data/parameter_sets_4glaciers_5000samples_200ensembles_20180727.nc
5000
../MCMC_data/parameter_sets_4glaciers_1000samples_200ensembles_20180727.nc
1000
../MCMC_data/parameter_sets_4glaciers_30000samples_200ensembles_20180727.nc
30000
../MCMC_data/parameter_sets_4glaciers_3000samples_200ensembles_20180727.nc
3000


In [134]:
glaciers = ['15.1006', '15.10059', '15.10075', '15.10079']
variabl = ['massbal']#, 'precfactor', 'ddfsnow', 'massbal']
samples.sort()
samples = ['1000', '3000', '5000', '10000', '30000']

In [141]:
def get_glacier_data(glacier_number):
            '''
            Returns the mass balance and error estimate for
            the glacier from David Shean's DEM data


            Parameters
            ----------
            glacier_number : float
                RGI Id of the glacier for which data is to be
                returned. Should be a number with a one or two
                digit component before the decimal place
                signifying glacier region, and 5 numbers after
                the decimal which represent glacier number.
                Example: 15.03733 for glacier 3733 in region 15


            Returns
            -------
            (tuple)
            massbal : float
                average annual massbalance over david sheans's
                dataset
            stdev : float
                estimate error (standard deviation) of measurement
            index : int
                index of glacier in csv file for debugging

            '''

            #convert input to float
            glacier_number = float(glacier_number)

            # upload csv file of DEM data and convert to
            # dataframe
            csv_path = '../DEMs/hma_mb_20171211_1343.csv'
            df = pd.read_csv(csv_path)

            # locate the row corresponding to the glacier
            # with the given RGIId number
            row = df.loc[round(df['RGIId'], 5) == glacier_number]

            # get massbalance, measurement error (standard
            # deviation) and index of the
            # glacier (index for debugging purposes)
            index = row.index[0]
            massbal = row['mb_mwea'][index]
            stdev = row['mb_mwea_sigma'][index]

            return massbal, stdev, index

In [153]:
np.mean(ensembles['30000']['15.10075'].sel(variables='massbal').values)

0.25761620229183041

In [148]:

for glac in glaciers:
    print(glac)
    mean, std, index = get_glacier_data(float(glac))
    print('observed mean, std:', mean, std, index)
    #for key, val in ensembles.items():
    for sample in samples:
        val = ensembles[sample]
        print(sample)
    
        for var in variabl:
            print(var)
            mean = np.mean(val[glac].sel(variables=var).values)
            std = np.std(val[glac].sel(variables=var).values)
            print('mean:', mean, 'std:', std)
        

15.1006
observed mean, std: -0.041 0.239 3
1000
massbal
mean: 0.0485333188266 std: 0.237568151107
3000
massbal
mean: 0.067505779678 std: 0.255946164863
5000
massbal
mean: 0.0580876731769 std: 0.250958968707
10000
massbal
mean: 0.0462780729628 std: 0.242817111123
30000
massbal
mean: 0.0564343342403 std: 0.251202397916
15.10059
observed mean, std: 0.074 0.237 2
1000
massbal
mean: 0.216274570474 std: 0.244817067067
3000
massbal
mean: 0.261092985526 std: 0.266327055556
5000
massbal
mean: 0.205778161922 std: 0.24982517005
10000
massbal
mean: 0.194019513758 std: 0.249969279872
30000
massbal
mean: 0.196155249536 std: 0.250257559106
15.10075
observed mean, std: 0.084 0.24 0
1000
massbal
mean: 0.296195645042 std: 0.260783993624
3000
massbal
mean: 0.204392290351 std: 0.254536689438
5000
massbal
mean: 0.241138062934 std: 0.254339059152
10000
massbal
mean: 0.274163802237 std: 0.264499436747
30000
massbal
mean: 0.257616202292 std: 0.263810390215
15.10079
observed mean, std: 0.082 0.238 1
1000
massb

In [105]:
ensembles['1000']['15.1006'].sel(variables='tempchange')

<xarray.DataArray '15.1006' (runs: 200)>
array([ 2.188107,  2.188107,  2.188107,  2.791304,  2.716221,  2.306758,
        2.306758,  2.320939,  2.320939,  2.676396,  2.458666,  2.684647,
        2.479075,  3.370174,  3.370174,  2.887615,  2.887615,  2.125781,
        2.125781,  0.899419,  2.477995,  2.478984,  3.252994,  3.252994,
        3.29834 ,  3.29834 ,  2.374229,  2.374229,  2.239797,  1.014241,
        2.869379,  1.472456,  2.244555,  2.029186,  2.029186,  2.642548,
        3.302095,  2.915113,  2.915113,  2.868803,  2.290042,  2.290042,
        2.290042,  2.836791,  2.836791,  2.764643,  2.764643,  2.764643,
        2.426777,  2.280555,  2.42327 ,  2.010638,  2.632609,  2.632609,
        2.097875,  4.202466,  3.760174,  3.760174,  3.718561,  2.772188,
        2.772188,  3.009554,  2.780785,  2.186985,  2.242949,  2.242949,
        2.242949,  2.90643 ,  3.477393,  3.477393,  3.626729,  3.626729,
        4.319518,  4.010175,  4.010175,  2.442332,  2.442332,  2.379803,
        0.